## Exercise 9 | data in f1ex9 | update the Land Use and Land Cover Map of 2018 (COS 2018) by adding information on areas affected by fires from 2019 to the present. Replace the class in the COS 2018 with the “Burned Areas” class in regions affected by fires since 2019.

In [ ]:
lulc = '/mnt/d/cwsig/exercicios_2023_24/f1ex9/cos2018.shp'

burns = [
    '/mnt/d/cwsig/exercicios_2023_24/f1ex9/aa_2019.shp',
    '/mnt/d/cwsig/exercicios_2023_24/f1ex9/aa_2020.shp',
    '/mnt/d/cwsig/exercicios_2023_24/f1ex9/aa_2021.shp',
    '/mnt/d/cwsig/exercicios_2023_24/f1ex9/ardida_2022.shp'
]

cols_values = {
    'ID' : 0,
    'COS18n1_C' : '10.9.9', 'COS18n1_L' : 'Burn Areas',
    'COS18n2_C' : '10.9.9', 'COS18n2_L' : 'Burn Areas',
    'COS18n3_C' : '10.9.9', 'COS18n3_L' : 'Burn Areas',
    'COS18n4_C' : '10.9.9', 'COS18n4_L' : 'Burn Areas',
    'Area_ha' : 0
}

olulc = '/mnt/d/cwsig/exercicios_2023_24/f1ex9/cos2018_up.shp'

In [ ]:
import geopandas as gp

from glass.rd.shp import shp_to_obj
from glass.dtt.mge.pd import merge_df
from glass.wt.shp import df_to_shp

In [ ]:
lulc_df = shp_to_obj(lulc)

burns_dfs = [shp_to_obj(s) for s in burns]

burn_df = merge_df(burns_dfs)

In [ ]:
# Intersect
ires = gp.overlay(lulc_df, burn_df, how='intersection')

# Dissolve
dres = ires.dissolve()

dc = []
for c in dres.columns.values:
    if c == 'geometry': continue

    if c in cols_values:
        dres[c] = cols_values[c]
    else:
        dc.append(c)

# Erase
nlulc = lulc_df.overlay(dres, how='difference')

if len(dc):
    nlulc.drop([c for c in nlulc.columns.values if c in dc], axis=1, inplace=True)
    dres.drop([c for c in dres.columns.values if c in dc], axis=1, inplace=True)

# Concat/Merge
flulc = merge_df([nlulc, dres])

In [ ]:
df_to_shp(flulc, olulc)